In [1]:
from pathlib import Path

from ray import tune
from ray.air import Checkpoint, session

In [2]:
ray_results = Path("~/ray_results").expanduser()
assert ray_results.is_dir()
last_experiment = max(ray_results.glob('_aux_train_daostack_*'), key=lambda x: x.stat().st_ctime)
print(f"Loading results from {last_experiment}...")
assert last_experiment.is_dir()

def _aux_train_daostack(*args, **kwargs):
    raise NotImplementedError

# TODO: Create an experiment grid or whatever is called
tuner = tune.Tuner.restore(str(last_experiment), _aux_train_daostack)
rg = tuner.get_results()
rg.get_best_result()

Loading results from /home/davo/ray_results/_aux_train_daostack_2023-08-16_13-31-22...


2023-08-16 13:36:29,337	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.
2023-08-16 13:36:29,356	WARNING experiment_analysis.py:916 -- Failed to read the results for 1 trials:
- /home/davo/ray_results/_aux_train_daostack_2023-08-16_13-31-22/_aux_train_daostack_f4570c1d_4_trial_index=3,batch_size=4,conv_layers=4,embedding_dim=175,l2=0.0001,learning_rate=0.0041,max_epoch_2023-08-16_13-31-31


Result(
  metrics={'loss': 0.18073842377233348, 'rprec train': 0.5126403570175171, 'rprec test': 0.5126403570175171, 'p@5 train': 0.6750000715255737, 'p@5 test': 0.6750000715255737, 'r@5 train': 0.11846227943897247, 'r@5 test': 0.11846227943897247, 'should_checkpoint': True, 'done': False, 'trial_id': '6624f00d', 'experiment_tag': '2_trial_index=1,batch_size=4,conv_layers=4,embedding_dim=175,l2=0.0001,learning_rate=0.0041,max_epochs=50'},
  path='/home/davo/ray_results/_aux_train_daostack_2023-08-16_13-31-22/_aux_train_daostack_6624f00d_2_trial_index=1,batch_size=4,conv_layers=4,embedding_dim=175,l2=0.0001,learning_rate=0.0041,max_epoch_2023-08-16_13-31-24',
  checkpoint=Checkpoint(local_path=/home/davo/ray_results/_aux_train_daostack_2023-08-16_13-31-22/_aux_train_daostack_6624f00d_2_trial_index=1,batch_size=4,conv_layers=4,embedding_dim=175,l2=0.0001,learning_rate=0.0041,max_epoch_2023-08-16_13-31-24/checkpoint_000011)
)

In [3]:
# Read dataframe and drop some unneeded columns
rdf = rg.get_dataframe('p@5 test', 'max').drop(columns=['timestamp', 'node_ip', 'pid', 'hostname', 'time_since_restore', 'iterations_since_restore', 'logdir'])
rdf = rdf.drop(columns=['date'])
# rdf = rdf[rdf['done']]
print(rdf.columns)

for c in ['config/batch_size']: #, 'config/embedding_dim']:
    rdf[c] = 2**rdf[c]

rdf.sort_values('loss').head(5)

2023-08-16 13:36:29,382	WARNING experiment_analysis.py:694 -- Failed to read the config for 1 trials:
- /home/davo/ray_results/_aux_train_daostack_2023-08-16_13-31-22/_aux_train_daostack_f4570c1d_4_trial_index=3,batch_size=4,conv_layers=4,embedding_dim=175,l2=0.0001,learning_rate=0.0041,max_epoch_2023-08-16_13-31-31


Index(['loss', 'rprec train', 'rprec test', 'p@5 train', 'p@5 test',
       'r@5 train', 'r@5 test', 'time_this_iter_s', 'should_checkpoint',
       'done', 'training_iteration', 'trial_id', 'time_total_s',
       'config/__trial_index__', 'config/batch_size', 'config/conv_layers',
       'config/embedding_dim', 'config/l2', 'config/learning_rate',
       'config/max_epochs'],
      dtype='object')


,loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,should_checkpoint,done,training_iteration,trial_id,time_total_s,config/__trial_index__,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,config/max_epochs
2,0.183580,0.506565,0.506565,0.667308,0.667308,0.117797,0.117797,24.184288,True,False,12,f3d1f5ed,286.050801,0,16,4,175,0.000064,0.00409,50
1,0.193042,0.516550,0.516550,0.686538,0.686538,0.137289,0.137289,24.220886,True,False,11,6624f00d,265.497123,1,16,4,175,0.000064,0.00409,50
0,0.194881,0.488606,0.488606,0.650000,0.650000,0.113907,0.113907,24.326813,True,False,10,30ac4776,244.658790,2,16,4,175,0.000064,0.00409,50


In [4]:
# Group by config (except trial index) and do the mean
non_config_columns = ['config/__trial_index__', 'config/max_epochs']
cfg_columns = [c for c in rdf.columns if c.startswith('config/') and c not in non_config_columns]
non_results_cols = ['should_checkpoint', 'done', 'config/__trial_index__'] + non_config_columns
rdfgb = rdf.drop(columns=non_results_cols).groupby(cfg_columns).mean(numeric_only=True).reset_index()
print(len(rdfgb), "configurations were tested")
rdfgb.sort_values('p@5 test', ascending=False).tail(30)

1 configurations were tested


,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,training_iteration,time_total_s
0,16,4,175,0.000064,0.00409,0.190501,0.503907,0.503907,0.667949,0.667949,0.122998,0.122998,24.243996,11.0,265.402238
